

# NBA Projections Scrubber 2
Author: Kahlil Gedin <br>
Date: 12/24/19 <br>
Revision History:<br>
    * (12/26/19) - std calculation was corrected and output.AvAvgPointsPerGame was set to player projections

## import modules and functions

In [1]:
import pandas as pd
import numpy as np
import math
import time

timestr =time.strftime("%d%m%y")

In [2]:
timestr

'250120'

In [3]:
#function to figure out probability 
def normpdf(x, mean, sd):
    """normpdf is the probability density function for a normal distribution. The output of this function
    is the probability of an event"""
    
    if sd != 0:
        var = float(sd)**2
        denom = (2*math.pi*var)**.5
        num = math.exp(-(float(x)-float(mean))**2/(2*var))
        return num/denom
    else:
        return 0

In [4]:
def FCcleaner(self):
    self.rename({"Player":"name"}, axis="columns", inplace=True)
    self.rename({"Salary":"salary"}, axis="columns", inplace=True)
    self.rename({"Team":"team"}, axis="columns", inplace=True)
    self.rename({"Pos":"pos"}, axis="columns", inplace=True)
    self.rename({"Opp":"opp"}, axis="columns", inplace=True)
    self.rename({"Ceiling":"ceil"}, axis="columns", inplace=True)
    self.rename({"Floor":"floor"}, axis="columns", inplace=True)
    self.rename({"FC Proj":"proj"}, axis="columns", inplace=True)
    self.rename({"ProjSTDV":"std"}, axis="columns", inplace=True)
    

In [5]:
def nameCorrector(data, mando):
    count = 0
    for i in range(len(data)):
        for j in range(len(mando)):
            if data.name.iloc[i] == mando.name.iloc[j]:
                data.name.iloc[i] = mando.DK.iloc[j]
                count = 1+count 
                
    return ("Changed {name} names".format(name=count))

In [6]:
def createSTDs(data):
    data['std']= ((data['ceil'].astype(float, copy = False)-data['floor'].astype(float, copy = False))/2)


def createODDs(data):
    data['odds']= data['pos']
    for i in range(len(data)):
        data['odds'].iloc[i] = normpdf(40,data['proj'].iloc[i],data['std'].iloc[i])*100

In [7]:
def ceFlo(data, ref):
    lump1 = dict(zip(ref.name, ref.ceil))
    lump2 = dict(zip(ref.name, ref.floor))
    lump3 = dict(zip(ref.name, ref.opp))
    lump4 = dict(zip(ref.name, ref.pos))
    lump5 = dict(zip(ref.name, ref.team))
    lump6 = dict(zip(ref.name, ref.salary))
    data['ceil'] = data.name.map(lump1)
    data['floor'] = data.name.map(lump2)
    data['opp'] = data.name.map(lump3)
    data['pos'] = data.name.map(lump4)
    data['team'] = data.name.map(lump5)
    data['salary'] = data.name.map(lump6)

In [8]:
def DFScleaner(self):
    self.rename({"Player":"name"}, axis="columns", inplace=True)
    self.rename({"Projection":"proj"}, axis="columns", inplace=True)

In [9]:
def ROBEcleaner(self):
    self.rename({"Name":"name"}, axis="columns", inplace=True)
    self.rename({"Sal":"sal"}, axis="columns", inplace=True)
    self.rename({"Team":"team"}, axis="columns", inplace=True)
    self.rename({"Pos":"pos"}, axis="columns", inplace=True)
    self.rename({"Opp":"opp"}, axis="columns", inplace=True)
    self.rename({"Proj":"proj"}, axis="columns", inplace=True)

## import data

In [10]:
# next step is to create a function to load in all the projections from one for loop

# importing Draftkings Salaries
dk = pd.read_csv('DKSalaries{date}.csv'.format(date=timestr))

In [11]:
# setup RG projections for use
rg = pd.read_csv('nba-player{date}.csv'.format(date=timestr), skiprows =[0], names = ['name', 'salary', 'team', 'pos', 'opp', 'ceil', 'floor', 'proj', 'own'])
nameCorrector(rg, pd.read_csv('NameIssues/RG.csv'))

createSTDs(rg)
createODDs(rg)

C:\Users\kyler\Anaconda4\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
rg[180:240]

,name,salary,team,pos,opp,ceil,floor,proj,own,std,odds


In [13]:
# setup FC projections for use
fc = pd.read_csv('draftkings_NBA_2020-01-17_players.csv', 
                 usecols=['Player', 'Salary', 'Team', 'Pos', 'Opp', 'Floor', 'Ceiling', 'FC Proj', 'ProjSTDV'],)
FCcleaner(fc)
nameCorrector(fc, pd.read_csv('NameIssues/FC.csv') )
createODDs(fc)

In [14]:
# setup DFS projections for use 
#dfs = pd.read_excel('Draftkings NBA ProjectionsJAN12.xlsx')
#nameCorrector(fc, pd.read_csv('NameIssues/DFS.csv') )
#DFScleaner(dfs)
#ceFlo(dfs, rg)

In [15]:
# setup robe projections for use
#robe = pd.read_csv('NBA Projections - PROJECTIONS15JAN20.csv', skiprows = [0, 1], usecols = ['Name',
#                                                                                          'Sal', 'Team', 'Pos',
#                    'Opp','Proj'])
#ROBEcleaner(robe)
#nameCorrector(fc, pd.read_csv('NameIssues/Robe.csv') )
#ceFlo(robe,rg)
#robe = robe[robe.team.notna()]

## combine DK ID numbers and salaries to projections

In [16]:
### packaging name and salary into dictionary
lump = dict(zip(dk.Name, dk.Salary))

# packaging name and name&ID into dictionary
lump2 = dict(zip(dk.Name, dk['Name + ID']))

# packaging name and ID into dictionary
lump3 = dict(zip(dk.Name, dk.ID))

# create new dataframe to store data
output = pd.DataFrame(columns = ['Position', 'Name + ID', 'Name', 'ID', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame'] )

In [17]:
data = rg

In [18]:
# setting position by player pool position
output['Position'] = data.pos

# setting nameid by Draftkings name ID
output['nameid'] = data.name.map(lump3)

# setting Name by player pool name
output['Name'] = data.name

# setting Name + ID by Draftkings Name + ID
output['Name + ID']= data.name.map(lump2)

# setting ID by Draftkings ID
output['ID'] = data.name.map(lump2)

# setting Salary by Draftkings Salary
output['Salary'] = data.name.map(lump)

# setting TeamAbbrev by player pool team abbreviation
output['TeamAbbrev'] = data.team

# setting AvgPointsPerGame by player pool projections
output['AvgPointsPerGame'] = data.proj

# removing lineups with NaN values
output= output[output.Salary.notna()]

## combine DK ID name to player pool

In [19]:
# creating a dictionary with outputs name and name + ID
lock = dict(zip(output.Name, output['Name + ID']))

# adding the Draftkings Name + ID to player pool
data['nameid'] = data.name.map(lock)

# removing player entries with NaN
data = data[data.nameid.notna()]

## save outputs

In [20]:
# saving lineup generator input
output.to_csv('output{date}.csv'.format(date=timestr))

In [21]:
output

,Position,Name + ID,Name,ID,Salary,TeamAbbrev,AvgPointsPerGame,nameid
0,PG,Darius Garland (14182036),Darius Garland,Darius Garland (14182036),5000.0,CLE,26.31,14182036.0
1,PG/SG,Ryan Arcidiacono (14182351),Ryan Arcidiacono,Ryan Arcidiacono (14182351),3000.0,CHI,8.92,14182351.0
2,C,Cristiano Felicio (14182109),Cristiano Felicio,Cristiano Felicio (14182109),4100.0,CHI,18.57,14182109.0
3,SF/PF,Chandler Hutchison (14182129),Chandler Hutchison,Chandler Hutchison (14182129),3900.0,CHI,17.23,14182129.0
4,PG/SG,Collin Sexton (14181973),Collin Sexton,Collin Sexton (14181973),6800.0,CLE,31.16,14181973.0
...,...,...,...,...,...,...,...,...
93,C,Andre Drummond (14181937),Andre Drummond,Andre Drummond (14181937),9100.0,DET,53.78,14181937.0
94,PF/C,Christian Wood (14182053),Christian Wood,Christian Wood (14182053),4700.0,DET,18.83,14182053.0
95,SG/SF,Svi Mykhailiuk (14182041),Svi Mykhailiuk,Svi Mykhailiuk (14182041),4900.0,DET,22.39,14182041.0
96,SF/PF,Sekou Doumbouya (14182079),Sekou Doumbouya,Sekou Doumbouya (14182079),4400.0,DET,19.16,14182079.0


In [22]:
# saving annotated player pool
data.to_csv('output2{date}.csv'.format(date=timestr))

In [23]:
data

,name,salary,team,pos,opp,ceil,floor,proj,own,std,odds,nameid
0,Darius Garland,5000,CLE,PG,CHI,34.7292,13.9443,26.31,14.40%,10.39245,1.61205,Darius Garland (14182036)
1,Ryan Arcidiacono,3000,CHI,PG/SG,CLE,13.2016,4.0140,8.92,0.09%,4.59380,9.97846e-10,Ryan Arcidiacono (14182351)
2,Cristiano Felicio,4100,CHI,C,CLE,23.9553,8.3565,18.57,0.09%,7.79940,0.11735,Cristiano Felicio (14182109)
3,Chandler Hutchison,3900,CHI,SF/PF,CLE,25.6727,7.7535,17.23,0.90%,8.95960,0.176247,Chandler Hutchison (14182129)
4,Collin Sexton,6800,CLE,PG/SG,CHI,43.6240,16.2032,31.16,1.80%,13.71040,2.36367,Collin Sexton (14181973)
...,...,...,...,...,...,...,...,...,...,...,...,...
93,Andre Drummond,9100,DET,C,BKN,70.9896,33.8814,53.78,48.59%,18.55410,1.6319,Andre Drummond (14181937)
94,Christian Wood,4700,DET,PF/C,BKN,25.4205,4.5192,18.83,0.81%,10.45065,0.490571,Christian Wood (14182053)
95,Svi Mykhailiuk,4900,DET,SG/SF,BKN,29.7787,11.1950,22.39,2.43%,9.29185,0.712614,Svi Mykhailiuk (14182041)
96,Sekou Doumbouya,4400,DET,SF/PF,BKN,26.8240,9.1968,19.16,0.09%,8.81360,0.276496,Sekou Doumbouya (14182079)
